<a href="https://colab.research.google.com/github/AUrieta13/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark

# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_Reviews").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|     650634| R3EQSTM9PWRAL|B0091LBZSU|     578484426|Demograss Capsule...|Health & Personal...|          3|            0|          0|   N|                Y|         Three Stars|Only came with 30...| 2015-08-31|
|         US|   19827510| RBWPRK17XKIXD|B00PWW3LQ6|     456433146|Viva Labs #1 Prem...|Health & Personal

In [5]:
df

DataFrame[marketplace: string, customer_id: int, review_id: string, product_id: string, product_parent: int, product_title: string, product_category: string, star_rating: int, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string, review_headline: string, review_body: string, review_date: string]

In [6]:
#Filter the data and create a new DataFrame retrieve rows where total_votes count is equal or greater than 20
totalvotes_df = df.filter("total_votes >= 20")
totalvotes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    1717317|R2984F9VD9WDJB|B0013OVVK0|     895291357|Jarrow Formulas N...|Health & Personal...|          1|           37|         46|   N|                Y|... month 2000 mg...|took it upon my d...| 2015-08-31|
|         US|   37405868|R218TGWCIAZYNY|B0065ZTKWS|     961894951|New Mama Bottom S...|Health & Personal

In [7]:
#Filter totalvotes_df and create new dataframe to retrieve rows where helpful_votes/total_votes is equal or greater to 50%
helpfulvotes_df = totalvotes_df.filter("helpful_votes/total_votes >= .50")
helpfulvotes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    1717317|R2984F9VD9WDJB|B0013OVVK0|     895291357|Jarrow Formulas N...|Health & Personal...|          1|           37|         46|   N|                Y|... month 2000 mg...|took it upon my d...| 2015-08-31|
|         US|   37405868|R218TGWCIAZYNY|B0065ZTKWS|     961894951|New Mama Bottom S...|Health & Personal

In [9]:
#Filter helpvotes_df and create dataframe that retrieves rows where review is vine == "Y"
Yvine_review_df = helpfulvotes_df.filter(helpfulvotes_df["vine"] == "Y")
Yvine_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   34746219| RG70K1HMY4LXX|B010H76PTU|     581545062|Omron 7 Series Wi...|Health & Personal...|          4|           37|         44|   Y|                N|Only tracks 1 per...|This product come...| 2015-08-27|
|         US|   51401810|R1DHGTNXDXJ0GB|B00VAPRSP4|     373707856|Cellucor, SuperHD...|Health & Personal

In [11]:
#Filter helpvotes_df and create dataframe that retrieves rows where review is vine == "N"
Nvine_review_df = helpfulvotes_df.filter(helpfulvotes_df["vine"] == "N")
Nvine_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    1717317|R2984F9VD9WDJB|B0013OVVK0|     895291357|Jarrow Formulas N...|Health & Personal...|          1|           37|         46|   N|                Y|... month 2000 mg...|took it upon my d...| 2015-08-31|
|         US|   37405868|R218TGWCIAZYNY|B0065ZTKWS|     961894951|New Mama Bottom S...|Health & Personal

Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid)


In [12]:
from pyspark.sql.functions import count

#total number of paid reviews
Yvine_count = Yvine_review_df.count()
print(Yvine_count)

#total number of unpaid reviews
Nvine_count = Nvine_review_df.count()
print(Nvine_count)

497
120863


In [13]:
#number of 5 star paid reviews
Y5_star_df = Yvine_review_df.filter(Yvine_review_df["star_rating"] == "5")
print(Y5_star_df.count())

#number of 5 star unpaid reviews
N5_star_df = Nvine_review_df.filter(Nvine_review_df["star_rating"] == "5")
print(N5_star_df.count())


220
74470


In [20]:
#Percentage of 5 Star Reviews paid reviews
star5_Yvine_percent = (Y5_star_df.count()/Yvine_count)*100
print("Percent of 5 star for paid reviews: %f" % star5_Yvine_percent)

#Percentage of 5 Star Reviews unpaid reviews
star5_Nvine_percent = (N5_star_df.count()/Nvine_count)*100
print("Percent of 5 star for unpaid reviews: %f" % star5_Nvine_percent)


Percent of 5 star for paid reviews: 44.265594
Percent of 5 star for unpaid reviews: 61.615217


In [24]:
fivestar_df = helpfulvotes_df.filter(df["star_rating"] == "5")
fivestar_df.count()

74690